# Extract

In [1]:
%load_ext jupyter_black

In [4]:
import os
import gzip
import shutil
from pathlib import Path
from datetime import datetime
from typing import Iterator, Literal

from requests import Session, HTTPError
import pandas as pd
import xarray as xr
import zarr
import asyncio
import aiohttp
import time

import mmmpy

Archive = Literal["zip", "tar", "gztar", "bztar", "xztar"]
data = Path(os.path.abspath(__name__)).parents[1] / "data"
data

/opt/venv/lib/python3.10/site-packages/yaml/parser.py:98: RuntimeWarning: coroutine 'main' was never awaited
  self.current_event = self.state()
<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 16 from C header, got 96 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject


PosixPath('/workspaces/MMM-Py/data')

In [5]:
from numpy.typing import NDArray
import numpy as np
from typing import Callable, Literal
GZ=".gz"

def __iter(url: str, session:Session, condition_callback:Callable[[pd.Series], NDArray[np.bool_]], ) -> Iterator[str]:
    r = session.get(url)
    r.raise_for_status()
    (html,) = pd.read_html(r.content)
    directory = html["Name"].dropna()
    yield from url + directory[condition_callback(directory)]

def main(parent = "3DRef"):
    baseurl = f"http://mrms.ncep.noaa.gov/data/{parent}/"
    # baseurl += parent 
        # ...
    # elif parent == "2D":
    #     ...
    # else:
        # ...
        # raise Exception
    # return baseurl
    # if not destination.exists():
    #     destination.mkdir()
    with Session() as session:
        session.headers.update([("accept", "html")])
        # iterating the first page provides the levels that are avaliable in the 3DRefl database
        return  tuple(__iter(baseurl, session, lambda s: s.str.contains("MergedReflectivityQC")))
        # for url in tuple(__iter(baseurl))[:1]:
if __name__ == "__main__":
    df = main()
df

HTTPError: HTTP Error 404: Not Found

In [25]:





def __iterlevels(baseurl: str) -> Iterator[str]:
    (html,) = pd.read_html(baseurl)
    levels = html["Name"].dropna()
    
    yield from baseurl + levels[levels.str.contains("MergedReflectivityQC")]


def __itertimes(url: str, input_dt: datetime, max_seconds: int) -> Iterator[str]:
    # all of the levels pages are read to get the validtimes to each of the files and file url
    (html,) = pd.read_html(url, skiprows=[1, 2, 3], parse_dates=True)
    # then some logic to select only recent files
    files = html["Name"].dropna()
    time_delta: pd.Series[datetime] = abs(
        input_dt - files.str.extract(r"(\d{8}-\d{6})").astype("datetime64[s]").squeeze()
    )
    yield from files[time_delta.dt.total_seconds() <= max_seconds]


def __make_archive(
    source: Path,
    destination: Path,
    archive_type: Archive = "gztar",
) -> None:
    base_name = destination.parent / destination.stem

    shutil.make_archive(
        str(base_name), archive_type, root_dir=source.parent, base_dir=source.name
    )

def from_ncep(
    destination: Path,
    *,
    input_dt: datetime = datetime.utcnow(),
    max_seconds: int = 300,
    archive: Archive = None,
    parent:Literal["3DRef","2D"]="3DRef",
    region:Literal["ALASKA", "CARIB", "GUAM", "HAWAII"]|None=None
    # headers=HEADERS,
) -> None:
    """
    baseurl = "http://mrms.ncep.noaa.gov/data/{parent}/"

    downloads files the the mrms dataset
    TODO: [SPECIFC PRODUCT SELECTION]
    -

    TODO: [MULTI-THREADING]
    - add an option for multi-threading to alow for simultaneous downloads

    """
    OPTIONS = {"3DRefl":("ALASKA", "CARIB", "GUAM", "HAWAII"), "2D":("ALASKA")}
    if parent not in OPTIONS or (region and region not in OPTIONS[parent]):
        raise Exception

    # if parent == "3DRef" and region in ("ALASKA", "CARIB", "GUAM", "HAWAII"):

    baseurl = f"http://mrms.ncep.noaa.gov/data/{parent}/{region if region else ''}"
    # baseurl += parent 
        # ...
    # elif parent == "2D":
    #     ...
    # else:
        # ...
        # raise Exception
    # return baseurl
    if not destination.exists():
        destination.mkdir()
    with Session() as session:
        session.headers.update([("accept", "gzip")])
        # iterating the first page provides the levels that are avaliable in the 3DRefl database
        for url in tuple(__iterlevels(baseurl))[:1]:
            # all of the levels pages are read to get the validtimes to each of the files and file url
            # then some logic to select only recent files
            for file in tuple(__itertimes(url, input_dt, max_seconds))[:1]:
                try:
                    # a request is made to hit the file url
                    r = session.get(url + file, stream=True)
                    r.raise_for_status()
                except HTTPError:
                    continue
                # the response object is decompressed
                with gzip.GzipFile(fileobj=r.raw, mode="rb") as fsrc:
                    file = destination / file.removesuffix(GZ)
                    # written to the local drive
                    with file.open("wb") as fdst:
                        shutil.copyfileobj(fsrc, fdst)
    if archive:
        # passing an archive argument will archive the files
        # this is useful for the git purposes
        __make_archive(destination, destination.with_suffix(f".{archive}"))

import uuid
if __name__ == "__main__":
    url = from_ncep(data / f"MRMS_TEST{uuid.uuid1()}", parent="3DRefl")
url

In [2]:
# async request
websites = """https://www.youtube.com
https://www.facebook.com
https://www.baidu.com
https://www.yahoo.com
https://www.amazon.com
https://www.wikipedia.org
http://www.qq.com
https://www.google.co.in
https://www.twitter.com
https://www.live.com
http://www.taobao.com
https://www.bing.com
https://www.instagram.com
http://www.weibo.com
http://www.sina.com.cn
https://www.linkedin.com
http://www.yahoo.co.jp
http://www.msn.com
http://www.uol.com.br
https://www.google.de
http://www.yandex.ru
http://www.hao123.com
https://www.google.co.uk
https://www.reddit.com
https://www.ebay.com
https://www.google.fr
https://www.t.co
http://www.tmall.com
http://www.google.com.br
https://www.360.cn
http://www.sohu.com
https://www.amazon.co.jp
http://www.pinterest.com
https://www.netflix.com
http://www.google.it
https://www.google.ru
https://www.microsoft.com
http://www.google.es
https://www.wordpress.com
http://www.gmw.cn
https://www.tumblr.com
http://www.paypal.com
http://www.blogspot.com
http://www.imgur.com
https://www.stackoverflow.com
https://www.aliexpress.com
https://www.naver.com
http://www.ok.ru
https://www.apple.com
http://www.github.com
http://www.chinadaily.com.cn
http://www.imdb.com
https://www.google.co.kr
http://www.fc2.com
http://www.jd.com
http://www.blogger.com
http://www.163.com
http://www.google.ca
https://www.whatsapp.com
https://www.amazon.in
http://www.office.com
http://www.tianya.cn
http://www.google.co.id
http://www.youku.com
https://www.example.com
http://www.craigslist.org
https://www.amazon.de
http://www.nicovideo.jp
https://www.google.pl
http://www.soso.com
http://www.bilibili.com
http://www.dropbox.com
http://www.xinhuanet.com
http://www.outbrain.com
http://www.pixnet.net
http://www.alibaba.com
http://www.alipay.com
http://www.chrome.com
http://www.booking.com
http://www.googleusercontent.com
http://www.google.com.au
http://www.popads.net
http://www.cntv.cn
http://www.zhihu.com
https://www.amazon.co.uk
http://www.diply.com
http://www.coccoc.com
https://www.cnn.com
http://www.bbc.co.uk
https://www.twitch.tv
https://www.wikia.com
http://www.google.co.th
http://www.go.com
https://www.google.com.ph
http://www.doubleclick.net
http://www.onet.pl
http://www.googleadservices.com
http://www.accuweather.com
http://www.googleweblight.com
http://www.answers.yahoo.com"""
import aiohttp
import asyncio

async def main():

    async with aiohttp.ClientSession() as session:
        async with session.get('http://python.org') as response:

            print("Status:", response.status)
            print("Content-type:", response.headers['content-type'])

            html = await response.text()
            print("Body:", html[:15], "...")


# if __name__ == "__main__":
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
    # urls = websites.split("\n")
    # start = time.time()
    # asyncio.run(main(urls))
    # end = time.time()


RuntimeError: This event loop is already running

In [2]:
files = sorted((data / "MRMS_MergedReflectivity").glob("*.grib2"))
files

[PosixPath('/workspaces/MMM-Py/data/MRMS_MergedReflectivity/MRMS_MergedReflectivityQC_00.50_20220820-095643.grib2'),
 PosixPath('/workspaces/MMM-Py/data/MRMS_MergedReflectivity/MRMS_MergedReflectivityQC_00.50_20220820-095839.grib2'),
 PosixPath('/workspaces/MMM-Py/data/MRMS_MergedReflectivity/MRMS_MergedReflectivityQC_00.75_20220820-095643.grib2'),
 PosixPath('/workspaces/MMM-Py/data/MRMS_MergedReflectivity/MRMS_MergedReflectivityQC_00.75_20220820-095839.grib2'),
 PosixPath('/workspaces/MMM-Py/data/MRMS_MergedReflectivity/MRMS_MergedReflectivityQC_01.00_20220820-095643.grib2'),
 PosixPath('/workspaces/MMM-Py/data/MRMS_MergedReflectivity/MRMS_MergedReflectivityQC_01.00_20220820-095839.grib2'),
 PosixPath('/workspaces/MMM-Py/data/MRMS_MergedReflectivity/MRMS_MergedReflectivityQC_01.25_20220820-095643.grib2'),
 PosixPath('/workspaces/MMM-Py/data/MRMS_MergedReflectivity/MRMS_MergedReflectivityQC_01.25_20220820-095839.grib2'),
 PosixPath('/workspaces/MMM-Py/data/MRMS_MergedReflectivity/MRMS

In [3]:
%%time
# passing a single grib file
mmmpy.read_mrms(files[0], engine="cfgrib")
# The mmmpy.MRMSDataset is a wrapper around the xarray.Dataset

CPU times: user 7.48 s, sys: 429 ms, total: 7.9 s
Wall time: 7.91 s


<xarray.DataArray 'MRMS_MergedReflectivityQC' (MRMS_MergedReflectivityQC: 1,
                                               heightAboveSea: 1,
                                               latitude: 3500, longitude: 7000)>
dask.array<stack, shape=(1, 1, 3500, 7000), dtype=float32, chunksize=(1, 1, 3500, 7000), chunktype=numpy.ndarray>
Coordinates:
  * heightAboveSea             (heightAboveSea) float64 500.0
  * latitude                   (latitude) float64 54.99 54.98 ... 20.02 20.01
  * longitude                  (longitude) float64 230.0 230.0 ... 300.0 300.0
  * MRMS_MergedReflectivityQC  (MRMS_MergedReflectivityQC) object 'MRMS_Merge...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             161
    GRIB_centreDescription:  161
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             161

In [4]:
%%time
# 4 files 
mrms = mmmpy.read_mrms(files[:4], engine="cfgrib")
mrms

CPU times: user 28.3 s, sys: 1.18 s, total: 29.5 s
Wall time: 29.6 s


<xarray.DataArray 'MRMS_MergedReflectivityQC' (MRMS_MergedReflectivityQC: 1,
                                               validTime: 2, heightAboveSea: 2,
                                               latitude: 3500, longitude: 7000)>
dask.array<stack, shape=(1, 2, 2, 3500, 7000), dtype=float32, chunksize=(1, 2, 1, 3500, 7000), chunktype=numpy.ndarray>
Coordinates:
  * validTime                  (validTime) datetime64[ns] 2022-08-20T09:56:00...
  * heightAboveSea             (heightAboveSea) float64 500.0 750.0
  * latitude                   (latitude) float64 54.99 54.98 ... 20.02 20.01
  * longitude                  (longitude) float64 230.0 230.0 ... 300.0 300.0
  * MRMS_MergedReflectivityQC  (MRMS_MergedReflectivityQC) object 'MRMS_Merge...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             161
    GRIB_centreDescription:  161
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             161

In [8]:
%%time
zarr_bucket = data / "zarr"
if zarr_bucket.exists():
    shutil.rmtree(zarr_bucket)
mrms.to_zarr(zarr_bucket)

CPU times: user 1.67 s, sys: 261 ms, total: 1.94 s
Wall time: 1.53 s


In [2]:
%%time
zarr_bucket = data / "zarr"
mmmpy.read_mrms(zarr_bucket, engine="zarr")
# reduced the read time from  30 s -> 42 ms 

CPU times: user 32.9 ms, sys: 10.8 ms, total: 43.6 ms
Wall time: 42 ms


<xarray.DataArray 'MRMS_MergedReflectivityQC' (MRMS_MergedReflectivityQC: 1,
                                               validTime: 2, heightAboveSea: 2,
                                               latitude: 3500, longitude: 7000)>
dask.array<stack, shape=(1, 2, 2, 3500, 7000), dtype=float32, chunksize=(1, 2, 1, 3500, 7000), chunktype=numpy.ndarray>
Coordinates:
  * heightAboveSea             (heightAboveSea) float64 500.0 750.0
  * latitude                   (latitude) float64 54.99 54.98 ... 20.02 20.01
  * longitude                  (longitude) float64 230.0 230.0 ... 300.0 300.0
  * validTime                  (validTime) datetime64[ns] 2022-08-20T09:56:00...
  * MRMS_MergedReflectivityQC  (MRMS_MergedReflectivityQC) object 'MRMS_Merge...
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             161
    GRIB_centreDescription:  161
    GRIB_edition:            2
    GRIB_subCentre:          0
    institution:             161